In [65]:
#import pandas as pd
!wget http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
!unzip dataset_tsmc2014.zip

--2024-05-29 18:28:06--  http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
Resolving www-public.tem-tsp.eu (www-public.tem-tsp.eu)... 157.159.10.107, 2001:660:3203:100:1:0:80:107
Connecting to www-public.tem-tsp.eu (www-public.tem-tsp.eu)|157.159.10.107|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25546284 (24M) [application/zip]
Saving to: ‘dataset_tsmc2014.zip.2’

dataset_tsmc2014.zi 100%[===================>]  24.36M  6.29MB/s    in 5.4s    

2024-05-29 18:28:12 (4.54 MB/s) - ‘dataset_tsmc2014.zip.2’ saved [25546284/25546284]

Archive:  dataset_tsmc2014.zip
replace dataset_tsmc2014/dataset_TSMC2014_NYC.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset_tsmc2014/dataset_TSMC2014_readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset_tsmc2014/dataset_TSMC2014_TKY.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [164]:
import pandas as pd
columns = ['User ID',
 'Venue ID',
 'Venue category ID',
 'Venue category name',
 'Latitude',
 'Longitude',
 'Timezone',
 'UTC time']

In [165]:
NUMCHECKIN = 7000

df = pd.read_csv('dataset_tsmc2014/dataset_TSMC2014_NYC.txt', sep='\t', encoding='latin-1', names=columns)
df = df.sample(NUMCHECKIN)
print(df)

        User ID                  Venue ID         Venue category ID  \
114249     1083  436bf600f964a520ab291fe3  4bf58dd8d48988d116941735   
204732      553  4b20ed51f964a5206c3524e3  4bf58dd8d48988d1e0931735   
219490      386  4e6a06081495acb0c68d2ca3  4bf58dd8d48988d13b941735   
60770       314  4c5b02d52091a5931fcc60d0  4bf58dd8d48988d12c941735   
52999       596  4fa7629de4b028d55b6dc3b7  4bf58dd8d48988d103941735   
...         ...                       ...                       ...   
120093      429  4bccba5e8920b71351a39edc  4bf58dd8d48988d1a7941735   
191100       64  427d5680f964a520a8211fe3  4bf58dd8d48988d18f941735   
8365       1044  4b5882e7f964a520665b28e3  4bf58dd8d48988d124941735   
195988      156  4af82bc2f964a520370b22e3  4bf58dd8d48988d1e0931735   
27545       592  4de05ecad22d2a79556daa01  4bf58dd8d48988d10a951735   

              Venue category name   Latitude  Longitude  Timezone  \
114249                        Bar  40.729872 -73.980802      -240   
204732   

In [166]:

checkin_counts = df['User ID'].value_counts() # Conta il numero di check-in per ogni utente
users_with_sufficient_checkins = checkin_counts[checkin_counts >= 5].index   # Filtra gli utenti con meno di 5 check-in
df = df[df['User ID'].isin(users_with_sufficient_checkins)]  # Mantieni solo le righe relative agli utenti con almeno 5 check-in

print(f"Numero di utenti con meno di 5 check-in: {len(checkin_counts[checkin_counts < 5])}")

Numero di utenti con meno di 5 check-in: 470


In [167]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
df['Venue ID'] = le.fit_transform(df['Venue ID'])
df['Venue category ID'] = le.fit_transform(df['Venue category ID'])
df.drop(['Venue category name'],axis=1,inplace=True)
print(df)

        User ID  Venue ID  Venue category ID   Latitude  Longitude  Timezone  \
114249     1083       228                 41  40.729872 -73.980802      -240   
204732      553      1187                222  40.708135 -73.830764      -300   
219490      386      3118                 95  40.802367 -74.155968      -300   
52999       596      3677                  6  40.851558 -74.141613      -240   
97820       222       847                203  40.765353 -73.918217      -240   
...         ...       ...                ...        ...        ...       ...   
26233        57      1850                140  40.799194 -74.098953      -240   
72935       484      1351                262  40.689905 -73.951777      -240   
120093      429      1807                176  40.597722 -74.148543      -240   
191100       64       199                158  40.671639 -73.963812      -300   
8365       1044      1368                 64  40.705892 -74.005703      -240   

                              UTC time 

<ipython-input-167-fa028e661b39>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Venue ID'] = le.fit_transform(df['Venue ID'])
<ipython-input-167-fa028e661b39>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Venue category ID'] = le.fit_transform(df['Venue category ID'])
<ipython-input-167-fa028e661b39>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [168]:
import math

def distance_between_coordinates(lat1, lon1, lat2, lon2):
    radius = 6371  # Radius of the Earth in km

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Calculate the differences in latitude and longitude<
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    # Calculate the distance using the Haversine formula
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance

In [169]:
print(distance_between_coordinates(40.719810,-74.002581,40.735981, -74.029309))

2.8819941850086326


In [170]:
#build an alternative temporal graph
#two pois are connected if users go on average to them in the seme timeframe

df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')

df['localDayofWeek'] = df.localTime.dt.dayofweek
df['localHour'] = df.localTime.dt.hour






<ipython-input-170-f85843e15d2c>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')


In [171]:
venuesdf = df[['Venue ID','Latitude','Longitude','Venue category ID']].drop_duplicates(subset=['Venue ID'])
numvenues = len(venuesdf)
print(venuesdf)


        Venue ID   Latitude  Longitude  Venue category ID
114249       228  40.729872 -73.980802                 41
204732      1187  40.708135 -73.830764                222
219490      3118  40.802367 -74.155968                 95
52999       3677  40.851558 -74.141613                  6
97820        847  40.765353 -73.918217                203
...          ...        ...        ...                ...
4796        1083  40.867481 -73.924285                159
71198       3680  40.703235 -73.943620                270
88782       1684  40.709174 -73.820701                262
191100       199  40.671639 -73.963812                158
8365        1368  40.705892 -74.005703                 64

[3895 rows x 4 columns]


In [172]:
!pip3 install torch --index-url https://download.pytorch.org/whl/${CUDA}
import torch
print(torch.__version__)
print(torch.version.cuda)
!pip install torch-geometric \
  torch-sparse \
  torch-scatter \
  torch-cluster \
  -f https://pytorch-geometric.com/whl/torch-2.3.0+cu121.html

Looking in indexes: https://download.pytorch.org/whl/
2.3.0+cu121
12.1
Looking in links: https://pytorch-geometric.com/whl/torch-2.3.0+cu121.html


In [173]:
import networkx as nx
venuesdict = venuesdf.to_dict('records')

i = 0
nodeIds = {}
node_attr = []

for el in venuesdict:
  #print(el['Venue ID'])
  nodeIds[el['Venue ID']] = i
  node_attr.append([float(el['Venue category ID'])])
  i+=1

print(len(node_attr))


3895


In [174]:
#import torch
sequences = {}
count = {}

MINVISIT = 5


for _, row in df.iterrows():

  if row['User ID'] not in sequences:
    sequences[row['User ID']] = [[row['Venue ID'],row['localHour'],row['localDayofWeek']]]


  else:
    sequences[row['User ID']].append([row['Venue ID'],row['localHour'],row['localDayofWeek']])


  if row['Venue ID'] not in count:
    count[row['Venue ID']] = 1
  else:
    count[row['Venue ID']] += 1

print(sequences)
seq = []
next = []
to_remove = []

for el in sequences:
  entry = [el]
  for i in range(len(sequences[el][len(sequences[el])-1])):
    el2 = sequences[el][len(sequences[el])-1][i]
    #print(el2,i)
    if i == 0:
      el2 = nodeIds[el2]
    #print(el2,i)
    entry.append(el2)
  for i in range(len(sequences[el])-1):
    entry.append(nodeIds[sequences[el][i][0]])
  seq.append(torch.tensor(entry))

  target = torch.zeros(numvenues)
  target[sequences[el][len(sequences[el])-1][0]] = 1
  next.append(target)
  #print(count[sequences[el][len(sequences[el])-1][0]])
  if count[sequences[el][len(sequences[el])-1][0]] < MINVISIT:

    to_remove.append(sequences[el][len(sequences[el])-1][0])
    #seq.append(torch.tensor(entry))
    #next.append(target)

seq=torch.nn.utils.rnn.pad_sequence(seq,batch_first=True)
seq_dim = len(seq[0])-4
print(seq_dim)
#seq=torch.nn.utils.rnn.pad_sequence(seq)

print(len(to_remove))

#to_remove_new = []

#for el in to_remove:

#  el = nodeIds[el]
#  to_remove_new[]

print(seq)
print(len(next))



{1083: [[228, 20, 2], [628, 13, 5], [628, 12, 4], [154, 1, 5], [2527, 22, 4], [571, 0, 6], [92, 20, 4], [628, 10, 1], [703, 12, 1], [628, 10, 0], [59, 20, 5]], 553: [[1187, 5, 1], [2250, 8, 3], [1749, 11, 5], [1414, 4, 5], [1020, 14, 5], [1187, 7, 1], [2542, 8, 2], [1020, 18, 2], [1414, 11, 4]], 386: [[3118, 8, 3], [1755, 19, 5], [1487, 5, 0], [3076, 8, 2], [3118, 9, 0], [3076, 8, 1], [3118, 8, 3], [3118, 7, 1], [1892, 9, 4], [3855, 12, 6], [3730, 15, 2], [3076, 12, 1]], 596: [[3677, 1, 0], [2534, 17, 3], [3677, 19, 2], [946, 18, 6], [974, 12, 0], [1991, 15, 6], [1178, 21, 2], [1510, 2, 4], [1677, 22, 6], [2103, 14, 5], [1340, 8, 0]], 222: [[847, 19, 2], [806, 11, 6], [313, 17, 3], [1387, 11, 6], [272, 15, 6], [2354, 14, 6]], 81: [[3472, 19, 4], [913, 19, 4], [376, 14, 3], [1270, 1, 1], [1638, 4, 5], [1270, 8, 2], [203, 8, 3], [1270, 8, 4], [1618, 20, 5], [1532, 19, 6], [1270, 8, 1], [96, 18, 5], [757, 1, 5], [1270, 2, 2], [90, 20, 3]], 143: [[1723, 19, 0], [1302, 15, 4], [1659, 17, 3]

In [175]:
df = df[~df['Venue ID'].isin(to_remove)]
venuesdf = venuesdf[~venuesdf['Venue ID'].isin(to_remove)]
venuesdict = venuesdf.to_dict('records')
print(df)
print(venuesdf)

        User ID  Venue ID  Venue category ID   Latitude  Longitude  Timezone  \
204732      553      1187                222  40.708135 -73.830764      -300   
219490      386      3118                 95  40.802367 -74.155968      -300   
52999       596      3677                  6  40.851558 -74.141613      -240   
97820       222       847                203  40.765353 -73.918217      -240   
185752       81      3472                 45  40.757084 -73.967946      -300   
...         ...       ...                ...        ...        ...       ...   
46340       323      1976                222  40.782618 -73.953349      -240   
33867       315      2339                100  40.672982 -73.870842      -240   
198458      881      3570                250  40.813338 -73.956249      -300   
106635      746       237                240  40.645089 -73.784523      -240   
213225      293      3809                282  40.689045 -73.930319      -300   

                              UTC time 

In [176]:
#fast spatial graph construction

import networkx as nx
venuesdict = venuesdf.to_dict('records')
#print(venuesdict)

#for el in venuesdict:
#  print((el,venuesdict[el]))
'''
G = nx.Graph()

distances = []

from tqdm import tqdm

for i in tqdm(range(len(venuesdict) - 1)):
    if not G.has_node(venuesdict[i]['Venue ID']):
        G.add_node(venuesdict[i]['Venue ID'], category=venuesdict[i]['Venue category ID'])

    for j in range(i + 1, len(venuesdict)):
        if not G.has_node(venuesdict[j]['Venue ID']):
            G.add_node(venuesdict[j]['Venue ID'], category=venuesdict[i]['Venue category ID'])

        if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                        venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
            G.add_edge(venuesdict[i]['Venue ID'], venuesdict[j]['Venue ID'])
'''
from tqdm import tqdm
import torch
from torch_geometric.data import Data

edge_index = []
edges_attr = []

for i in tqdm(range(len(venuesdict) - 1)):

   #if venuesdict[i]['Venue ID'] not in node_attr:
   #  node_attr.append(venuesdict[i]['Venue ID'])

   for j in range(i + 1, len(venuesdict)):
    #  if venuesdict[j]['Venue ID'] not in node_attr:
    #     node_attr.append(venuesdict[j]['Venue ID'])


      if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                    venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
        edge_index.append([nodeIds[venuesdict[i]['Venue ID']], nodeIds[venuesdict[j]['Venue ID']]])
        edges_attr.append([1])

spatG = Data(x=torch.tensor(node_attr), edge_index=torch.LongTensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(spatG)



100%|██████████| 3359/3359 [00:18<00:00, 180.20it/s]


Data(x=[3895, 1], edge_index=[2, 134321], edge_attr=[134321, 1])


In [177]:
import pickle

print(spatG)

#spatpG = G

pickle.dump(spatG,open('spatG.pickle','wb'))

#data = pickle.load(open('spatG.pickle','rb'))

Data(x=[3895, 1], edge_index=[2, 134321], edge_attr=[134321, 1])


In [178]:
#nx.draw(G)
print(spatG)

Data(x=[3895, 1], edge_index=[2, 134321], edge_attr=[134321, 1])


In [179]:

#sequantial relation graph
#build a graph that captures the sequence of actions made by the users

import networkx as nx

edge_index = []
edges_attr = []

usrloc = {}

#seqG = nx.Graph()

for _, row in df.iterrows():

  #if not seqG.has_node(row['Venue ID']):
  #  seqG.add_node(row['Venue ID'], category=row['Venue category ID'])

  if row['User ID'] not in usrloc:
    usrloc[row['User ID']] = row['Venue ID']

  else:
    #seqG.add_edge(usrloc[row['User ID']],row['Venue ID'])
    edge_index.append([nodeIds[usrloc[row['User ID']]], nodeIds[row['Venue ID']]])
    edges_attr.append(1)
    usrloc[row['User ID']] = row['Venue ID']


seqG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(seqG)






Data(x=[3895, 1], edge_index=[2, 4224], edge_attr=[4224])


In [180]:
import pickle


print(seqG)

pickle.dump(seqG,open('tempG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

Data(x=[3895, 1], edge_index=[2, 4224], edge_attr=[4224])


In [181]:
import math
groupeddf = df.groupby(['Venue ID','Venue category ID'],as_index=False)
avghour = groupeddf['localHour'].mean()
avgday = groupeddf['localDayofWeek'].mean()

print(avghour)

timedf = pd.concat([avghour,avgday['localDayofWeek']],axis=1)



print(timedf)

hours = {}
week = {}

#week['workdays'] = []
#week['weekend'] = []


for _, row in timedf.iterrows():

  hour = math.floor(row['localHour'])
  day = math.floor(row['localDayofWeek'])

  if hour not in hours:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    hours[hour] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    hours[hour].append([row['Venue ID'],row['Venue category ID']])

  #if 1 <= day <=4:
  #  week['workdays'].append([row['Venue ID'],row['Venue category ID']])
  #else:
  #  week['weekend'].append([row['Venue ID'],row['Venue category ID']])

  if day not in week:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    week[day] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    week[day].append([row['Venue ID'],row['Venue category ID']])






      Venue ID  Venue category ID  localHour
0            0                 52        0.0
1            1                233       21.0
2            2                 41       22.0
3            3                 41       21.0
4            5                208       21.0
...        ...                ...        ...
3367      3890                302       16.0
3368      3891                170       13.0
3369      3892                 58        9.0
3370      3893                212       12.0
3371      3894                123       21.0

[3372 rows x 3 columns]
      Venue ID  Venue category ID  localHour  localDayofWeek
0            0                 52        0.0             6.0
1            1                233       21.0             6.0
2            2                 41       22.0             3.0
3            3                 41       21.0             2.0
4            5                208       21.0             4.0
...        ...                ...        ...             ...
3367    

In [182]:
#dayG = nx.Graph()
#hourG = nx.Graph()

edge_index = []
edges_attr = []

for el in hours:
   for u in hours[el]:

    #if not hourG.has_node(u[0]):
    #  hourG.add_node(u[0], category=u[1])

    for v in hours[el]:

      if u != v:

        #if not hourG.has_node(u[0]):
        #  hourG.add_node(v[0], category=v[1])

        #hourG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

hourG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(hourG)




Data(x=[3895, 1], edge_index=[2, 580830], edge_attr=[0])


In [183]:
pickle.dump(hourG,open('hourG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

In [184]:
edge_index = []
edges_attr = []


for el in week:
   for u in week[el]:

    #if not dayG.has_node(u[0]):
    #  dayG.add_node(u[0], category=u[1])

    for v in week[el]:

      if u != v:

        #if not dayG.has_node(u[0]):
        #  dayG.add_node(v[0], category=v[1])

        #dayG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

dayG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(dayG)

Data(x=[3895, 1], edge_index=[2, 1647804], edge_attr=[0])


In [185]:
pickle.dump(dayG,open('dayG.pickle','wb'))

In [186]:
!pip install pytorch-lightning
import pytorch_lightning as pl

In [187]:
import matplotlib.pyplot as plt
import torch
from sklearn.manifold import TSNE
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

from torch_geometric.datasets import Planetoid
from torch_geometric.nn import Node2Vec

# Lightning Module for Node2Vec model
class Node2VecModel(pl.LightningModule):
    def __init__(self, data, embedding_dim=32, walk_length=20, context_size=10, walks_per_node=10, num_negative_samples=1, p=1.0, q=1.0, lr=0.01):
        super().__init__()
        self.data = data
        self.model = Node2Vec(
            data.edge_index,
            embedding_dim=embedding_dim,
            walk_length=walk_length,
            context_size=context_size,
            walks_per_node=walks_per_node,
            num_negative_samples=num_negative_samples,
            p=p,
            q=q,
            sparse=True
        )
        self.lr = lr

    def forward(self):
        return self.model()

    def training_step(self, batch, batch_idx):
        pos_rw, neg_rw = batch
        loss = self.model.loss(pos_rw.to(self.device), neg_rw.to(self.device))
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.SparseAdam(self.parameters(), lr=self.lr)

    def test_step(self, batch, batch_idx):
        z = self.model()
        acc = self.model.test(
            train_z=z[self.data.train_mask],
            train_y=self.data.y[self.data.train_mask],
            test_z=z[self.data.test_mask],
            test_y=self.data.y[self.data.test_mask],
            max_iter=150
        )
        self.log('test_acc', acc)
        return acc




batch_size = 128
num_workers = 4
graph_embed = []

for el in [spatG,hourG,seqG,dayG]:

  model = Node2VecModel(data=el)

  loader = model.model.loader(batch_size=128, shuffle=True, num_workers=num_workers)

  trainer = pl.Trainer(
      max_epochs=1,
      callbacks=[
          ModelCheckpoint(monitor='train_loss'),
          EarlyStopping(monitor='train_loss', patience=10)
      ]
  )
  trainer.fit(model, loader)
  #trainer.test(model, loader)

  # Plotting points


  model.eval()
  z = model().detach()
  graph_embed.append(z)

  #z = TSNE(n_components=2).fit_transform(z)
  print(z.shape)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type     | Params
-----------------------------------
0 | model | Node2Vec | 124 K 
-----------------------------------
124 K     Trainable params

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type     | Params
-----------------------------------
0 | model | Node2Vec | 124 K 
-----------------------------------
124 K     Trainable params
0         Non-trainable params
124 K     Total params
0.497     Total estimated model params size (MB)


torch.Size([3879, 32])


Training: |          | 0/? [00:00<?, ?it/s]

...

torch.Size([3879, 32])


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type     | Params
-----------------------------------
0 | model | Node2Vec | 124 K 
-----------------------------------
124 K     Trainable params
0         Non-trainable params
124 K     Total params
0.497     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


torch.Size([3879, 32])


In [188]:
from sklearn.model_selection import train_test_split

batchsize = 32

#train_x,test_x,train_y,test_y = train_test_split(seq,torch.stack(next),test_size=0.2,stratify=torch.stack(next))
train_x,test_x,train_y,test_y = train_test_split(seq,torch.stack(next),test_size=0.2)
#train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2,stratify=train_y)
train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2)

trainset = torch.utils.data.TensorDataset(train_x,train_y)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batchsize,shuffle=True)
print(len(trainloader))

valset = torch.utils.data.TensorDataset(val_x,val_y)
valloader = torch.utils.data.DataLoader(valset,batch_size=batchsize,shuffle=True)
print(len(valloader))

testset = torch.utils.data.TensorDataset(test_x,test_y)
testloader = torch.utils.data.DataLoader(testset,batch_size=batchsize,shuffle=True)
print(len(testloader))

12
3
4


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import pytorch_lightning as pl

class GCN(pl.LightningModule):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class LightningModel(pl.LightningModule):
    def __init__(self, input_dim, location_dim, sequence_dim, graph_embed):
        super(LightningModel, self).__init__()
        self.graph_embed = graph_embed
        self.train_losses = []
        self.val_losses = []
        # User Embedding Layer
        self.user_embedding = torch.nn.Embedding(num_embeddings=input_dim, embedding_dim=32)

        # Last location Embedding Layer
        self.lastLoc_embedding = torch.nn.Embedding(num_embeddings=input_dim, embedding_dim=32)

        # Time and day Input linear
        self.timeday_embedding = torch.nn.Linear(2, 32)

        # Sequence Embedding Layer
        self.self_attn = torch.nn.MultiheadAttention(embed_dim=sequence_dim, num_heads=1)

        # Expand sequence representation
        self.expand_sequence = torch.nn.Linear(sequence_dim, 32)

        # Multihead Attention Layer
        self.multihead_attn = torch.nn.MultiheadAttention(embed_dim=32, num_heads=2)

        # Classifier Layers
        self.linear1 = torch.nn.Linear(int((4*32*(self.graph_embed[0].shape[0]+batchsize))/batchsize), 128)
        self.linear2 = torch.nn.Linear(128, 64)
        self.linear3 = torch.nn.Linear(64, numvenues)

    def forward(self, user, lastloc, lasthour, lastday, sequence):
        user_embed = self.user_embedding(user.int())
        lastloc_embed = self.lastLoc_embedding(lastloc.int())
        timeday_embed = self.timeday_embedding(torch.stack([lasthour.float(), lastday.float()], dim=1))
        graph_embed = torch.cat(self.graph_embed)

        # Sequence Embedding
        sequence_embed, _ = self.self_attn(sequence.float(), sequence.float(), sequence.float())
        sequence_embed = self.expand_sequence(sequence_embed)

        # Concatenate all embeddings
        concat_embed = torch.cat([user_embed, lastloc_embed, graph_embed.to(self.device), timeday_embed, sequence_embed], dim=0)

        # Multihead Attention
        attn_output, _ = self.multihead_attn(concat_embed, concat_embed, concat_embed)

        # Classifier
        out = F.relu(self.linear1(attn_output.view(batchsize, -1)))
        out = F.relu(self.linear2(out))
        out = self.linear3(out)

        return out

    def training_step(self, batch, batch_idx):

        x, y = batch
        pad_dim = batchsize - x.shape[0]
        if pad_dim != 0:
            x = torch.cat([x, torch.zeros(pad_dim, x.shape[1]).to(self.device)])
            y = torch.cat([y, torch.zeros(pad_dim, y.shape[1]).to(self.device)])
        y_pred = self(x[:, 0], x[:, 1], x[:, 2], x[:, 3], x[:, 4:])
        loss = F.cross_entropy(torch.sigmoid(y_pred), y)
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        self.train_losses.append(loss.item())
        return loss


    def validation_step(self, batch, batch_idx):
            x, y = batch
            pad_dim = batchsize - x.shape[0]
            if pad_dim != 0:
                x = torch.cat([x, torch.zeros(pad_dim, x.shape[1]).to(self.device)])
                y = torch.cat([y, torch.zeros(pad_dim, y.shape[1]).to(self.device)])
            y_pred = self(x[:, 0], x[:, 1], x[:, 2], x[:, 3], x[:, 4:])
            val_loss = F.cross_entropy(y_pred, y)
            acc_at_k = accuracy_at_k(y_pred, y, k=5)
            mrr = mean_reciprocal_rank(y_pred, y)

            self.log('val_loss', val_loss, on_step=False, on_epoch=True)
            self.val_losses.append(val_loss.item())
            self.log('val_acc_at_5', acc_at_k, on_step=False, on_epoch=True)
            self.log('val_mrr', mrr, on_step=False, on_epoch=True)
            return val_loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def train_dataloader(self):
        return trainloader

    def val_dataloader(self):
            return valloader

def accuracy_at_k(y_pred, y_true, k=5):
    _, top_k = y_pred.topk(k, dim=1)
    _,label = y_true.topk(1, dim=1)

    print(top_k)
    print(label)

    correct = top_k.eq(label.view(-1, 1).expand_as(top_k))
    #print(label.view(-1, 1).expand_as(top_k))
    acc_at_k = correct.float().sum(dim=1).mean()
    print(acc_at_k)
    return acc_at_k

def mean_reciprocal_rank(y_pred, y_true):
    _, rank = y_pred.sort(dim=1, descending=True)
    rank = rank.argsort(dim=1)
    _,label = y_true.topk(1, dim=1)
    #print(label.view(-1, 1))
    rr = (1.0 / (rank.gather(1, label.view(-1, 1).long()) + 1)).mean()
    print(rr)
    return rr



# Esempio di addestramento del modello
input_dim = 10000
location_dim = 32
sequence_dim = seq_dim
model = LightningModel(input_dim, location_dim, sequence_dim,graph_embed)

trainer = pl.Trainer(max_epochs=1000, log_every_n_steps=1,callbacks=[
          ModelCheckpoint(monitor='train_loss'),
          EarlyStopping(monitor='train_loss', patience=10)
      ])
trainer.fit(model, trainloader,valloader)

train_losses = model.train_losses
val_losses = model.val_losses

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
train_losses = model.train_losses
val_losses = model.val_losses

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()